In [22]:
import pandas as pd
import time
import random

from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

import pygame
from pygame import mixer

bebidas = {
    "Bebidas refrescantes": "https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=",
    "Cervezas":"https://www.alcampo.es/compra-online/bebidas/cervezas/c/W1107?q=%3Arelevance&page=",
    "Vino Tinto":"https://www.alcampo.es/compra-online/bebidas/vino-tinto/c/W1151?q=%3Arelevance&page=",
    # "Zumos de Frutas":"https://www.alcampo.es/compra-online/bebidas/zumos-de-frutas/c/W1102?q=%3Arelevance&page=",
    # "Bebidas Alcohólicas":"https://www.alcampo.es/compra-online/bebidas/bebidas-alcoholicas/c/W1154?q=%3Arelevance&page=",
    # "Vino Blanco": "https://www.alcampo.es/compra-online/bebidas/vino-blanco/c/W1152?q=%3Arelevance&page=",
    # "Agua, Soda y Gaseosas": "https://www.alcampo.es/compra-online/bebidas/agua-soda-y-gaseosas/c/W1101?q=%3Arelevance&page=",
    # "Licores": "https://www.alcampo.es/compra-online/bebidas/licores/c/W1155?q=%3Arelevance&page=",
    # "Vino rosados, frizzantes, dulces y olorosos":"https://www.alcampo.es/compra-online/bebidas/vino-rosados-frizzantes-dulces-y-olorosos/c/W1153?q=%3Arelevance&page=",
    # "Champagne Cavas y Sidras": "https://www.alcampo.es/compra-online/bebidas/champagne-cavas-y-sidras/c/W1156?q=%3Arelevance&page=",
}


dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in bebidas.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                try:
                    dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                except:
                    dict_alcampo["Name"].append("Unkown")

                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

#Play sound when completed
mixer.init()
mixer.music.load(".././assets/completed.mp3")
mixer.music.set_volume(0.4)
mixer.music.play()  

df_alcampo




Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=0
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=1
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=2
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=3
Category: 192 Name: 192 Image: 192 Price: 192 Stock 192
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=4
Category: 240 Name: 240 Image: 240 Price: 240 Stock 240
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=5
Category: 288 Name: 288 Imag

,Category,Name,Price,Stock,Image
0,Bebidas refrescantes,Refresco de cola Zero sin azúcar COCA COLA lat...,"0,85 €(2,57 €/Litro)",Yes,https://www.alcampo.es/media/h43/h12/109888341...
1,Bebidas refrescantes,Refresco de cola Zero sin azúcar y sin cafeína...,"0,85 €(2,57 €/Litro)",Yes,https://www.alcampo.es/media/hec/h1c/111868914...
2,Bebidas refrescantes,Refresco de cola COCA COLA lata de 33 cl.,"0,85 €(2,57 €/Litro)",Yes,https://www.alcampo.es/media/hda/h23/111868986...
3,Bebidas refrescantes,Refresco de cola Zero sin azúcar PRODUCTO ALCA...,"0,69 €(0,34 €/Litro)",Yes,https://www.alcampo.es/media/h14/hbf/112700158...
4,Bebidas refrescantes,Refresco de limón zero FANTA ZERO lata de 33 cl.,"0,62 €(1,87 €/Litro)",Yes,https://www.alcampo.es/media/h1a/had/102076633...
...,...,...,...,...,...
1142,Vino Tinto,Vino tinto con denominación de origen califica...,"6,32 €",Yes,https://www.alcampo.es/media/h7c/hdd/111165202...
1143,Vino Tinto,Vino tinto gran reserva con denominación de or...,"344,24 €",Yes,https://www.alcampo.es/media/h9e/hb9/894040345...
1144,Vino Tinto,Vino tinto reserva con denominación de origen ...,"74,99 €",Yes,https://www.alcampo.es/media/hae/h35/890129435...
1145,Vino Tinto,Vino tinto reserva con denominación de origen ...,NaN,Yes,https://www.alcampo.es/media/hc0/hc8/118216319...


In [23]:
df_alcampo['Shop'] = 'Alcampo'

today = pd.Timestamp.now()
df_alcampo['Date'] = today
df_alcampo

df_alcampo.to_csv("../data/alcampo.csv", mode='a', index=False, header=False)

# read_file = pd.read_csv ("../data/alcampo.csv")
# read_file.to_excel ("../data/alcampo.xlsx", index = None, header=True)

alcampo = pd.read_csv("../data/alcampo.csv")
print(today)
alcampo

2023-03-18 13:08:07.320681


,Category,Name,Price,Stock,Image,Shop,Date
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...,Alcampo,2023-03-08 19:51:25.159831
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...,Alcampo,2023-03-08 19:51:25.159831
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...,Alcampo,2023-03-08 19:51:25.159831
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...,Alcampo,2023-03-08 19:51:25.159831
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...,Alcampo,2023-03-08 19:51:25.159831
...,...,...,...,...,...,...,...
198532,Vino Tinto,Vino tinto con denominación de origen califica...,"6,32 €",Yes,https://www.alcampo.es/media/h7c/hdd/111165202...,Alcampo,2023-03-18 13:08:07.320681
198533,Vino Tinto,Vino tinto gran reserva con denominación de or...,"344,24 €",Yes,https://www.alcampo.es/media/h9e/hb9/894040345...,Alcampo,2023-03-18 13:08:07.320681
198534,Vino Tinto,Vino tinto reserva con denominación de origen ...,"74,99 €",Yes,https://www.alcampo.es/media/hae/h35/890129435...,Alcampo,2023-03-18 13:08:07.320681
198535,Vino Tinto,Vino tinto reserva con denominación de origen ...,NaN,Yes,https://www.alcampo.es/media/hc0/hc8/118216319...,Alcampo,2023-03-18 13:08:07.320681
